# データの読み込み

In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob

#出力する行数を設定、省略をなくす
pd.set_option('display.max_rows', 500)

#データをまとめて読み込み、訓練データはtrain
train_files = glob.glob('./data/train/*.csv')
train = pd.DataFrame()
for file in train_files:
    train1 = pd.read_csv(file)
    cat_train = pd.concat([train,train1])
    train = cat_train

train.head()

/var/folders/ks/17djw9_d5r3300_0z8cc33sh0000gn/T/ipykernel_12992/1329853670.py:14: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  train1 = pd.read_csv(file)
/var/folders/ks/17djw9_d5r3300_0z8cc33sh0000gn/T/ipykernel_12992/1329853670.py:14: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  train1 = pd.read_csv(file)
/var/folders/ks/17djw9_d5r3300_0z8cc33sh0000gn/T/ipykernel_12992/1329853670.py:14: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  train1 = pd.read_csv(file)
/var/folders/ks/17djw9_d5r3300_0z8cc33sh0000gn/T/ipykernel_12992/1329853670.py:14: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  train1 = pd.read_csv(file)


,ID,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,...,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
0,40020197,中古マンション等,NaN,40133,福岡県,福岡市中央区,谷,桜坂,7,２ＬＤＫ,...,NaN,NaN,NaN,第１種中高層住居専用地域,60.0,150.0,2021年第2四半期,改装済,NaN,7.041393
1,40031380,中古マンション等,NaN,40137,福岡県,福岡市早良区,西新,西新,4,１Ｋ,...,NaN,NaN,NaN,商業地域,80.0,400.0,2020年第2四半期,未改装,NaN,6.602060
2,40030436,中古マンション等,NaN,40137,福岡県,福岡市早良区,小田部,室見,18,４ＬＤＫ,...,NaN,NaN,NaN,第１種中高層住居専用地域,60.0,150.0,2020年第4四半期,改装済,NaN,7.397940
3,40015295,中古マンション等,NaN,40132,福岡県,福岡市博多区,竹丘町,雑餉隈,5,２ＬＤＫ,...,NaN,NaN,NaN,商業地域,80.0,400.0,2020年第1四半期,未改装,NaN,7.278754
4,40147771,中古マンション等,NaN,40133,福岡県,福岡市中央区,荒戸,大濠公園,6,３ＤＫ,...,NaN,NaN,NaN,商業地域,80.0,400.0,2007年第3四半期,未改装,NaN,6.929419


## カラムの確認

In [99]:
train.columns

Index(['ID', '種類', '地域', '市区町村コード', '都道府県名', '市区町村名', '地区名', '最寄駅：名称',
       '最寄駅：距離（分）', '間取り', '面積（㎡）', '土地の形状', '間口', '延床面積（㎡）', '建築年', '建物の構造',
       '用途', '今後の利用目的', '前面道路：方位', '前面道路：種類', '前面道路：幅員（ｍ）', '都市計画', '建ぺい率（％）',
       '容積率（％）', '取引時点', '改装', '取引の事情等', '取引価格（総額）_log'],
      dtype='object')

# データの前処理

In [100]:
#目的変数
"""
'取引価格（総額）_log'
"""

#説明変数
"""
'ID', '市区町村コード', '都道府県名', '地区名', '最寄駅：名称', '最寄駅：距離（分）', '間取り', '面積（㎡）',
        '建築年', '建物の構造', '用途', '今後の利用目的', '都市計画', '建ぺい率（％）', '容積率（％）', '改装'
"""

###欠損値の削除###
class Delete_Null_Data():
    """
    欠損率が高いcolumnを削除するclass
    """
    
    def __init__(self,df):
        self.df = df
        
    #nullのカウントとnullの%のDataframeを返すメソッド
    def null_counts(self):
        null_counts_df = self.df
        null_counts_df = pd.DataFrame(null_counts_df.isnull().sum(),columns=['null_counts'])
        null_counts_df['null_par'] = null_counts_df['null_counts']/len(self.df)
        return null_counts_df
    
    #nullの多いcolumnを削除するメソッド引数にnullの%を指定　１００％→１　５０％→0.5
    def delete_null_columns(self,null_par=0.9):
        df = self.null_counts()
        re_df = df[df['null_par']>null_par]
        cols = re_df.index.tolist()
        delete_df = self.df.drop(labels=cols,axis=1)
        
        return delete_df


In [103]:
#インスタンス化して訓練データを更新する
Adj_Data=Delete_Null_Data(train)
train=Adj_Data.delete_null_columns()
train.shape

(722574, 20)

In [104]:
#trainデータの型確認
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 722574 entries, 0 to 37378
Data columns (total 20 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   ID            722574 non-null  int64  
 1   種類            722574 non-null  object 
 2   市区町村コード       722574 non-null  int64  
 3   都道府県名         722574 non-null  object 
 4   市区町村名         722574 non-null  object 
 5   地区名           721906 non-null  object 
 6   最寄駅：名称        719843 non-null  object 
 7   最寄駅：距離（分）     699417 non-null  object 
 8   間取り           696682 non-null  object 
 9   面積（㎡）         722574 non-null  object 
 10  建築年           701593 non-null  object 
 11  建物の構造         703422 non-null  object 
 12  用途            651713 non-null  object 
 13  今後の利用目的       356654 non-null  object 
 14  都市計画          702756 non-null  object 
 15  建ぺい率（％）       698413 non-null  float64
 16  容積率（％）        698413 non-null  float64
 17  取引時点          722574 non-null  object 
 18  改装   

In [105]:
'''
---データの前処理を施すメソッド---
名前：　preprocess
引数：　データフレーム
戻り値：　前処理をしたデータフレーム
'''

def preprocess(df):
    #欠損値補完で使うrandomをインポート
    import random
    
    #前処理を施す前のデータを確認
    print('==========Before preprocessing:==========',df.info())
    
    #種類, 市区町村名は削除------------------------------------<
    df=df.drop(['種類','市区町村名'], axis=1)
    
    #カテゴリ変数化------------------------------------<
    use_columns=["都道府県名", "地区名", "最寄駅：名称", "間取り", "建物の構造", "用途", "今後の利用目的", "都市計画", "改装"]
    for col in use_columns:
        df[col] = df[col].astype("category")
    
    
    #最寄駅：距離（分） object型→int型に変換------------------------------------<
    #時間がわからない部分の補完、欠損値は10で補完
    re={
        '30分?60分':'45', 
        '1H30?2H':'105', 
        '1H?1H30':'75', 
        '2H?':'120'
    }
    df['最寄駅：距離（分）']=df['最寄駅：距離（分）'].replace(re)
    df['最寄駅：距離（分）']=df['最寄駅：距離（分）'].fillna('10')
    df['最寄駅：距離（分）']=df['最寄駅：距離（分）'].astype(int)
    
    
    #面積（㎡） object型→int型(単位なし)に変換------------------------------------<
    #面積の欠損値はなし、'2000㎡以上'は'2000'で補完
    df['面積（㎡）']=df['面積（㎡）'].replace('2000㎡以上','2000')
    df['面積（㎡）']=df['面積（㎡）'].astype(int)
    
    
    #建築年 object型→int型(築年数)に変換------------------------------------<
    #欠損値を補完, nanのままだとうまくいかないので文字列でnullと入れておく
    df['建築年']=df['建築年'].fillna('null')
    
    l_k=[]
    now=2022
    y=df['建築年']
    
    for i in range(0,len(y),1):        
        #年号が令和、令和2年=2018+2=2020年
        if y.iloc[i][:2]=='令和':
            if y.iloc[i][3]=='年':
                year=now-(2018+int(y.iloc[i][2]))
            else:
                year=now-(2018+int(y.iloc[i][2])*10+int(y.iloc[i][3])) 
            l_k.append(year)

        
        #年号が平成、平成2年=1988+2=1991年
        elif y.iloc[i][:2]=='平成':
            if y.iloc[i][3]=='年':
                year=now-(1988+int(y.iloc[i][2]))
            else:
                year=now-(1988+int(y.iloc[i][2])*10+int(y.iloc[i][3])) 
            l_k.append(year)
            

        #年号が昭和、昭和2年=1925+2=1927年
        elif y.iloc[i][:2]=='昭和':
            if y.iloc[i][3]=='年':
                year=now-(1925+int(y.iloc[i][2]))
            else:
                year=now-(1925+int(y.iloc[i][2])*10+int(y.iloc[i][3])) 
            l_k.append(year)

        #年号が戦前、1868年から1945年の値をランダムに格納
        #1945年に終戦
        elif y.iloc[i][:2]=='戦前':
            year=now-random.randint(1868, 1945)
            l_k.append(year)

        #年号が欠損、1868年から2021年の値をランダムに格納
        else:
            #year=now-random.randint(1868, 2021)
            year=now-np.median(l_k)
            l_k.append(year)
    #リストを代入       
    df['建築年']=l_k
    
    #取引時点 object型→int型に変換------------------------------------<
    l_t=[]

    y1=df['取引時点']
    
    for i in range(0,len(y1),1):        
        #第1四半期
        if y1.iloc[i][6]=='1':
            torihiki=int(y1.iloc[i][:4])+0.25
            l_t.append(torihiki)

        #第2四半期
        elif y1.iloc[i][6]=='2':
            torihiki=int(y1.iloc[i][:4])+0.50
            l_t.append(torihiki)
            
        #第3四半期
        elif y1.iloc[i][6]=='3':
            torihiki=int(y1.iloc[i][:4])+0.50
            l_t.append(torihiki)
            
        #第4四半期
        else:
            torihiki=int(y1.iloc[i][:4])+0.99
            l_t.append(torihiki)
    #リストを代入        
    df['取引時点']=l_t
    
    #前処理を施した後のデータを確認
    print('==========After preprocessing:==========',df.info())
    
    
    #前処理後のデータを返す
    return df

In [106]:
#前処理した結果を更新して訓練データとして扱う
train=preprocess(train)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 722574 entries, 0 to 37378
Data columns (total 20 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   ID            722574 non-null  int64  
 1   種類            722574 non-null  object 
 2   市区町村コード       722574 non-null  int64  
 3   都道府県名         722574 non-null  object 
 4   市区町村名         722574 non-null  object 
 5   地区名           721906 non-null  object 
 6   最寄駅：名称        719843 non-null  object 
 7   最寄駅：距離（分）     699417 non-null  object 
 8   間取り           696682 non-null  object 
 9   面積（㎡）         722574 non-null  object 
 10  建築年           701593 non-null  object 
 11  建物の構造         703422 non-null  object 
 12  用途            651713 non-null  object 
 13  今後の利用目的       356654 non-null  object 
 14  都市計画          702756 non-null  object 
 15  建ぺい率（％）       698413 non-null  float64
 16  容積率（％）        698413 non-null  float64
 17  取引時点          722574 non-null  object 
 18  改装   

# モデルの構築(light GBM)

In [109]:
#必要なライブラリのインポート
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
#import lightgbm as lgb
#import optuna.integration.lightgbm as lgb


#説明変数 ： X
X=train.drop('取引価格（総額）_log', axis=1)
#目的変数 : y
y=train['取引価格（総額）_log']


In [ ]:
import lightgbm as lgb

#訓練データと検証データに分割, 7:3
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.3, random_state=1)

trains = lgb.Dataset(train_X, train_y)
valids = lgb.Dataset(val_X, val_y)

params = {
    'objective': 'regression',
    'metrics': 'mae',
    'n_estimators': 1000,
    'max_depth': 10,
    'boosting_type': 'gbdt',  # default = 'gbdt'
    'num_leaves': 400,         # default = 31,
    'learning_rate': 0.03,    # default = 0.1
    'feature_fraction': 0.95,  # default = 1.0
    'bagging_freq': 0,        # default = 0
    'bagging_fraction': 0.8,  # default = 1.0
    'random_state': 42        # default = None
}

model = lgb.train(params, trains, valid_sets=valids, num_boost_round=10000, early_stopping_rounds=100, verbose_eval=1000)

/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.021340 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11990
[LightGBM] [Info] Number of data points in the train set: 505801, number of used features: 17


/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Info] Start training from score 7.227813
Training until validation scores don't improve for 100 rounds


In [ ]:
import optuna

class Objective:
    def __init__(self, X, y):
        # 変数X,yの初期化
        self.X = X
        self.y = y

    def __call__(self, trial):
        # ハイパーパラメータの設定
        params = {
            # 最適化アルゴリズムを指定
            'solver' : trial.suggest_categorical('solver', ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']),
            # 正則化の強さを指定（0.0001から10まで）
            'C': trial.suggest_loguniform('C', 0.0001, 10),
            # 最大反復回数（＊ソルバーが収束するまで）
            'max_iter': trial.suggest_int('max_iter', 100, 100000)
            
            'max_depth': 
            }

        lgb_reg = lgb.LGBMRegressor(**params)

        # 評価指標として正解率の最大化を目指す
        scores = cross_validate(model,
                                X=self.X, 
                                y=self.y,
                                scoring='accuracy', 
                                n_jobs=-1) 
        return scores['test_score'].mean()

# ハイパーパラメータの探索
objective = Objective(X_train, y_train)
study = optuna.create_study(direction='maximize') # 最大化
study.optimize(objective, timeout=60)

# ベストパラメータを出力
print('params:', study.best_params)

In [94]:
"""
[ホールドアウト法]
・訓練用データと検証用データに分割
・訓練用データで学習、グリッドサーチでハイパーパラメータチューニング
・検証用データで予測、平均絶対誤差:MAEで精度確認
"""
#訓練データと検証データに分割, 7:3
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.3, random_state=1)

#モデルのインスタンス化
lgb_reg=lgb.LGBMRegressor(
                                                    #max_depth=5,
                                                    learning_rate=0.1,
                                                    n_estimators=5000,
                                                    #num_leaves=16
                                                )

#ハイパーパラメータの列挙
params = {
    #'n_estimators': [i for i in range(5000,10001, 1000)],
    'max_depth': [i for i in range(2,4)],
    'num_leaves': [2**i for i in range(2,5)]           
}

fit_params = {"early_stopping_rounds": 10,
                         "eval_set": [[val_X, val_y]]}

#ハイパーパラメータを探索して精度を比較する
grid=GridSearchCV(estimator=lgb_reg, 
                                  param_grid=params, 
                                  cv=5, 
                                  scoring = 'neg_mean_absolute_error',
                                  #fit_params=fit_params,
                                  verbose=1000
                                 )

grid.fit(train_X, train_y)

#探索の結果、最も高精度なハイパーパラメータ
print('ベストスコア：　', grid.best_score_)
print('最適ハイパーパラメータ：',grid.best_params_)


lgb_reg.fit(train_X, train_y)


Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5; 1/6] START max_depth=2, num_leaves=4...................................
[CV 1/5; 1/6] END ...max_depth=2, num_leaves=4;, score=-0.082 total time= 1.5min
[CV 2/5; 1/6] START max_depth=2, num_leaves=4...................................
[CV 2/5; 1/6] END ...max_depth=2, num_leaves=4;, score=-0.081 total time= 1.6min
[CV 3/5; 1/6] START max_depth=2, num_leaves=4...................................
[CV 3/5; 1/6] END ...max_depth=2, num_leaves=4;, score=-0.082 total time= 1.6min
[CV 4/5; 1/6] START max_depth=2, num_leaves=4...................................
[CV 4/5; 1/6] END ...max_depth=2, num_leaves=4;, score=-0.081 total time= 1.5min
[CV 5/5; 1/6] START max_depth=2, num_leaves=4...................................
[CV 5/5; 1/6] END ...max_depth=2, num_leaves=4;, score=-0.082 total time= 1.6min
[CV 1/5; 2/6] START max_depth=2, num_leaves=8...................................
[CV 1/5; 2/6] END ...max_depth=2, num_leaves=8;, 

KeyboardInterrupt: 

In [97]:
"""
[ホールドアウト法]
・訓練用データと検証用データに分割
・訓練用データで学習、グリッドサーチでハイパーパラメータチューニング
・検証用データで予測、平均絶対誤差:MAEで精度確認
"""
#訓練データと検証データに分割, 7:3
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.3, random_state=1)

#モデルのインスタンス化
lgb_reg=lgb.LGBMRegressor()

"""
#ハイパーパラメータの列挙
params = {
    #'n_estimators': [i for i in range(5000,10001, 1000)],
    'max_depth': [i for i in range(2,4)],
    'num_leaves': [2**i for i in range(2,5)]           
}

fit_params = {"early_stopping_rounds": 10,
                         "eval_set": [[val_X, val_y]]}

#ハイパーパラメータを探索して精度を比較する
grid=GridSearchCV(estimator=lgb_reg, 
                                  param_grid=params, 
                                  cv=5, 
                                  scoring = 'neg_mean_absolute_error',
                                  #fit_params=fit_params,
                                  verbose=1000
                                 )

grid.fit(train_X, train_y)

#探索の結果、最も高精度なハイパーパラメータ
print('ベストスコア：　', grid.best_score_)
print('最適ハイパーパラメータ：',grid.best_params_)
"""

lgb_reg.fit(train_X, train_y)


/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/optuna/integration/_lightgbm_tuner/sklearn.py:45: UserWarning: LightGBMTuner doesn't support sklearn API. Use `train()` or `LightGBMTuner` for hyperparameter tuning.
  warnings.warn(


RuntimeError: scikit-learn estimators should always specify their parameters in the signature of their __init__ (no varargs). <class 'optuna.integration._lightgbm_tuner.sklearn.LGBMRegressor'> with constructor (self, *args: Any, **kwargs: Any) -> None doesn't  follow this convention.

In [86]:
lgb_reg.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': 3,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': 'warn',
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [34]:
?GridSearchCV

In [82]:
#モデルのインスタンス化
lgb_reg=lgb.LGBMRegressor(n_estimators=10000,learning_rate=0.01,num_leaves=4,max_depth=2)

lgb_reg.fit(train_X, train_y, early_stopping_rounds=100, eval_set=(val_X,val_y), verbose=1000)

#精度の確認
print('平均絶対誤差 : ', mean_absolute_error(lgb_reg.predict(val_X), val_y))

/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/Users/Shota-0128/opt/anaconda3/lib/python3.9/s

[1000]	valid_0's l2: 0.0310794
[2000]	valid_0's l2: 0.0258579
[3000]	valid_0's l2: 0.0235849
[4000]	valid_0's l2: 0.022172
[5000]	valid_0's l2: 0.0212406
[6000]	valid_0's l2: 0.0206386
[7000]	valid_0's l2: 0.0201325
[8000]	valid_0's l2: 0.0197513
[9000]	valid_0's l2: 0.0194403
[10000]	valid_0's l2: 0.0191894
平均絶対誤差 :  0.08784953361975237


In [66]:
#精度の確認
print('平均絶対誤差 : ', mean_absolute_error(lgb_reg.predict(train_X), train_y))

平均絶対誤差 :  0.06145536965919432


In [67]:
#精度の確認
print('平均絶対誤差 : ', mean_absolute_error(lgb_reg.predict(X), y))

平均絶対誤差 :  0.06134046499274465


In [60]:
lgb_reg.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 5000,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': 'warn',
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

# テストデータで予測

In [61]:
#テストデータ読み込み
test=pd.read_csv('./data/test.csv')

#提出用サンプルファイル読み込み
sample=pd.read_csv('./data/sample_submission.csv')

#欠損値の多いカラムを除去
#インスタンス化して訓練データを更新する
Adj_Data=Delete_Null_Data(test)
test=Adj_Data.delete_null_columns()

test.head()


,ID,種類,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装
0,1000078,中古マンション等,1101,北海道,札幌市中央区,大通西,西１１丁目,1,３ＬＤＫ,75,平成28年,ＲＣ,住宅,住宅,商業地域,80.0,600.0,2021年第3四半期,未改装
1,1000121,中古マンション等,1101,北海道,札幌市中央区,大通西,西１８丁目,0,１ＤＫ,30,昭和52年,ＳＲＣ,住宅,住宅,商業地域,80.0,400.0,2021年第3四半期,改装済
2,1000123,中古マンション等,1101,北海道,札幌市中央区,大通西,西１８丁目,3,３ＬＤＫ,70,昭和59年,ＳＲＣ,住宅,住宅,商業地域,80.0,400.0,2021年第3四半期,改装済
3,1000127,中古マンション等,1101,北海道,札幌市中央区,大通西,西１８丁目,2,２ＬＤＫ,50,昭和64年,ＳＲＣ,住宅,住宅,商業地域,80.0,400.0,2021年第4四半期,未改装
4,1000129,中古マンション等,1101,北海道,札幌市中央区,大通西,西１８丁目,2,１ＤＫ,45,平成3年,ＳＲＣ,住宅,住宅,商業地域,80.0,400.0,2021年第4四半期,改装済


In [62]:
#テストデータ前処理
test=preprocess(test)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21005 entries, 0 to 21004
Data columns (total 19 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         21005 non-null  int64  
 1   種類         21005 non-null  object 
 2   市区町村コード    21005 non-null  int64  
 3   都道府県名      21005 non-null  object 
 4   市区町村名      21005 non-null  object 
 5   地区名        21004 non-null  object 
 6   最寄駅：名称     21000 non-null  object 
 7   最寄駅：距離（分）  21000 non-null  object 
 8   間取り        19847 non-null  object 
 9   面積（㎡）      21005 non-null  int64  
 10  建築年        20404 non-null  object 
 11  建物の構造      19776 non-null  object 
 12  用途         15396 non-null  object 
 13  今後の利用目的    20081 non-null  object 
 14  都市計画       20701 non-null  object 
 15  建ぺい率（％）    20596 non-null  float64
 16  容積率（％）     20596 non-null  float64
 17  取引時点       21005 non-null  object 
 18  改装         18032 non-null  object 
dtypes: float64(2), int64(3), object(14)
memory usa

In [68]:
###学習データの予測結果###
pred=lgb_reg.predict(test)

#提出用ファイルに予測結果格納(LightGBM)
sample['取引価格（総額）_log']=pred
sample.to_csv('submission10.csv', index=None)
